<a href="https://colab.research.google.com/github/mehrshadhosseini28/digikala_quera_2020/blob/master/DigiKala_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download File

In [ ]:
import os, requests

url = 'https://quera.ir/assignment/20120/download_problem_initial_project/66274/'
fname = "/content/P1.zip"


if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
else:
  print("this file is exist: ")

## extract zip

In [ ]:
import zipfile
with zipfile.ZipFile('/content/ML_User_Data.zip', 'r') as zip_ref:
    zip_ref.extractall("/content/")

### imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from sklearn import ensemble

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

# P1

### X_train,y_train

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Digikala/P1/train.csv')
date = np.array(data['date'])
month = [int(i.split('-')[1]) for i in date]
data['month'] = month

week = [datetime.datetime.strptime(data['date'][i],'%Y-%m-%d').weekday() for i in tqdm(range(len(data['date'])))]
data['week'] = week
print(data)

X_train = np.array(data[['seller','item','month','week']])
print('X=\n',X_train,X_train.shape)

y_train = np.array(data['sales'])
print('y=\n',y_train,y_train.shape)

### X_test

In [ ]:
test_data = pd.read_csv('/content/drive/My Drive/Digikala/P1/test.csv')

test_date = np.array(test_data['date'])
month = [int(i.split('-')[1]) for i in test_date]
test_data['month'] = month

week = [datetime.datetime.strptime(test_data['date'][i],'%Y-%m-%d').weekday() for i in tqdm(range(len(test_data['date'])))]
test_data['week'] = week
print(test_data)

X_test = np.array(test_data[['seller','item','month','week']])
print('X_test=\n',X_test,X_test.shape)

### Scale

In [ ]:
X = np.concatenate((X_train,X_test))
print(X.shape)
scaler = StandardScaler()
scl = scaler.fit(X)
X = scl.transform(X)
print(X,X.shape)

X_train = X[:900000]
X_test = X[900000:]
print(X_train.shape)
print(X_test.shape)

### Regressor

In [ ]:
clf = ensemble.GradientBoostingRegressor(n_estimators=1000 , max_depth=7, min_samples_split=2, verbose=1)
clf.fit(X_train,y_train)

clf.score(X_train,y_train)

### Test

In [ ]:
y_test_pred = clf.predict(X_test)

In [ ]:
id = ['id']
for i in range(len(y_test_pred)):
  id.append(i)

sales = ['sales']
for item in y_test_pred:
  item_int = round(item)
  sales.append(item_int)

In [ ]:
import csv

with open('/content/A1.csv', mode='w') as file:
    file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow([id[0],sales[0]])
    for i,j in zip(id[1:],sales[1:]):
      file_writer.writerow([str(int(i)),str(int(j))])